This Python program automatically extracts payment information (date of payment, gross pay, taxes, and net pay) from several PDF files stored in a specified directory. In order to extract the pertinent data, it first searches the directory for PDFs. Then, it parses the PDF contents for predetermined patterns. Next, the extracted data is compared with payment_info.xlsx, an Excel spreadsheet. The software avoids adding duplicate entries if the combination of the payment date and gross pay already exists in the spreadsheet; if not, it adds the new information to the spreadsheet. This procedure makes sure that, depending on payment information, the Excel file retains unique entries. As the program runs, it gives users feedback on how each PDF file is processed and informs them of any data additions or skips brought about by duplication avoidance.


In [1]:
import pandas as pd
import PyPDF2 as pyf
import pathlib
from pathlib import *

In [2]:
#Pegar o caminho
path_payslip = pathlib.Path.cwd()
print(path_payslip)
#descobre os arquivos
payslip_files = path_payslip.iterdir()
#cria uma lista com os arquivos
files_list = [file.name for file in payslip_files]
print(files_list)

#for para rodar arquivos
for file in files_list:
    if 'Payslip' in file:
        #read the archive
        reader_file = pyf.PdfReader(file)
        extracted_reader = reader_file.pages[0].extract_text()
        #pegar a primeira posicao
        posicao_inicial = extracted_reader.find('349W\n')
        #substituir os pontos por virugula
        extracted_reader = extracted_reader.replace('.',',')

        # pegar as posicções iniciais e finais
        inicial_data = extracted_reader.find('/', posicao_inicial) -2
        final_data = inicial_data + 10
        inicial_gross_pay = extracted_reader.find(',',final_data)-3
        final_gross_pay = extracted_reader.find(',',inicial_gross_pay)+3
        inicial_taxes = final_gross_pay + 1
        final_taxes = extracted_reader.find(',',inicial_taxes)+3
        inicial_net_pay = final_taxes + 1
        final_net_pay = extracted_reader.find(',',inicial_net_pay)+3

        #pegar os valores
        data_pagamento = extracted_reader[inicial_data:final_data]
        Gross_pay = extracted_reader[inicial_gross_pay:final_gross_pay]
        taxes = extracted_reader[inicial_taxes:final_taxes]
        net_pay = extracted_reader[inicial_net_pay:final_net_pay]
        #print(f'The payment data is {data_pagamento}')
        #print(f'The Gross pay is {Gross_pay}')
        #print(f'The taxes are {taxes}')
        #print(f'The net pay is {net_pay}')
        
        # Nova data a adicionar
        new_data = {
            'Payment Date': data_pagamento,
            'Gross Pay': Gross_pay,
            'Taxes': taxes,
            'Net Pay': net_pay
        }

        # o caminho do arquivo
        excel_path = Path('payment_info.xlsx')

        # Descobre se o arquivo ja existe
        if excel_path.exists():
            # Abre o arquivo caso exista
            existing_df = pd.read_excel(excel_path)
            # Verificar se a data e o grosspay estao juntos na minha linha, se sim, nao adiciona
            row_exists = existing_df[
                    (existing_df['Payment Date'] == data_pagamento) & 
                    (existing_df['Gross Pay'] == Gross_pay)
                ].any(axis=None)
           #verifica se existe o valor na planilha
            if row_exists:
                print("Payslip ja adicionado")
            else:
                # Adiciona o valor na planilha
                updated_df = existing_df.append(new_data, ignore_index=True)
                # Save the updated DataFrame back to the Excel file
                updated_df.to_excel(excel_path, index=False)
                print("Nova data adicionada")
        else:
            # Create a new DataFrame if the file does not exist
            updated_df = pd.DataFrame([new_data])
            # Save the updated DataFrame back to the Excel file
            updated_df.to_excel(excel_path, index=False)
            print("Nova planilha adicionada e valor adicionado")

             
print("\n \n Código finalizado")

c:\Users\User\Documents\GitHub\Get_payslips
['.git', '.gitattributes', 'Arquivo_final.ipynb', 'Pegar_payslips.ipynb']

 
 Código finalizado


I won't include the actual files due to containing personal information, but I'll provide the Excel file